# Tracking with bells and whistles

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sportslabkit as slk
from sportslabkit.logger import show_df # This just makes the df viewable in the notebook.


dataset_path = slk.datasets.get_path("top_view")
path_to_csv = sorted(dataset_path.glob("annotations/*.csv"))[0]
path_to_mp4 = sorted(dataset_path.glob("videos/*.mp4"))[0]

bbdf = slk.load_df(path_to_csv)

bbdf.head()

TeamID             0                                                   \
PlayerID           0                                        1           
Attributes bb_height bb_left   bb_top bb_width conf bb_height bb_left   
frame                                                                   
1               39.0  801.00  1051.00     39.0  1.0      39.0  1464.0   
2               39.0  801.75  1052.25     39.0  1.0      39.0  1464.0   
3               39.0  802.50  1053.50     39.0  1.0      39.0  1464.0   
4               39.0  803.25  1054.75     39.0  1.0      39.0  1464.0   
5               39.0  804.00  1056.00     39.0  1.0      39.0  1464.0   

TeamID                                 ...         1               \
PlayerID                               ...         9                
Attributes       bb_top bb_width conf  ... bb_height      bb_left   
frame                                  ...                          
1           1080.000000     39.0  1.0  ...      39.0  2537.000000   
2           1079.692308     39.0  1.0  ...      39.0  2536.857143   
3           1079.384615     39.0  1.0  ...      39.0  2536.714286   
4           1079.076923     39.0  1.0  ...      39.0  2536.571429   
5           1078.769231     39.0  1.0  ...      39.0  2536.428571   

TeamID                                     BALL                             \
PlayerID                                   BALL                              
Attributes       bb_top bb_width conf bb_height  bb_left   bb_top bb_width   
frame                                                                        
1           1146.000000     39.0  1.0     10.00  1916.00  1073.00    10.00   
2           1145.714286     39.0  1.0      9.75  1910.75  1073.75     9.75   
3           1145.428571     39.0  1.0      9.50  1905.50  1074.50     9.50   
4           1145.142857     39.0  1.0      9.25  1900.25  1075.25     9.25   
5           1144.857143     39.0  1.0      9.00  1895.00  1076.00     9.00   

TeamID           
PlayerID         
Attributes conf  
frame            
1           1.0  
2           1.0  
3           1.0  
4           1.0  
5           1.0  

[5 rows x 115 columns]

To use the full soccertrack dataset, see ["Dataset Preparation"](../02_user_guide/dataset_preparation.ipynb).

## Tracking setup

In [14]:
from sportslabkit.utils import get_git_root
from sportslabkit.mot import TeamTracker
from sportslabkit.mot.callbacks import TeamClassificationCallback

root = get_git_root()
cam = slk.Camera(path_to_mp4)

det_model = slk.detection_model.load('YOLOv8x', model=root / 'models/yolov8/soccer_top_view_best.pt', imgsz=1280)
motion_model = slk.motion_model.load('GCLinear')
calibration_model = slk.calibration_model.load('LineBasedCalibrator', min_line_length=50, line_distance_threshold=50, line_thickness=15, morph_size=15)
image_model = slk.image_model.load('resnet50')

first_matching_fn = slk.matching.MotionVisualMatchingFunction(
    motion_metric=slk.metrics.EuclideanCMM2D(use_pred_pt=True),
    motion_metric_gate=0.9,
    visual_metric=slk.metrics.CosineCMM(),
    visual_metric_gate=0.5,
    beta=0.9,
)

second_matching_fn = slk.matching.SimpleMatchingFunction(
    metric=slk.metrics.EuclideanCMM2D(use_pred_pt=True),
    gate=0.5,
)

vector_model = slk.vector_model.load('sklearnvectormodel', model_path='https://drive.google.com/file/d/1d6SskM1VFc_kIjFbQqEKgnMrmVnDRwWZ/view')
callbacks = [TeamClassificationCallback(vector_model)]

tracker = TeamTracker(
    detection_model=det_model,
    image_model=image_model,
    motion_model=motion_model,
    calibration_model=calibration_model,
    first_matching_fn=first_matching_fn,
    second_matching_fn=second_matching_fn,
    detection_score_threshold=0.5,
    max_staleness=3,
    min_length=5,
    callbacks=callbacks,
)

res = tracker.track(cam[:10])

Tracking Progress: 100%|██████████| 10/10 [01:02<00:00,  6.25s/it, Active: 9, Dead: 0]


In [9]:
save_path = "assets/tracking_results_with_teamtrack.mp4"
res.visualize_frames(cam.video_path, save_path)

Writing video: 10it [00:00, 10.83it/s]


In [17]:
codf = bbdf.to_codf(tracker.homographies[0], method='bottom_middle')
codf.head()

TeamID              0                                                         \
PlayerID            0                     1                    10              
Attributes          x          y          x          y          x          y   
frame                                                                          
1           13.153587  34.164146  36.983471  35.303577  53.197151  34.666561   
2           13.179957  34.208954  36.983601  35.292648  53.099606  34.701454   
3           13.206327  34.253761  36.983730  35.281727  53.002056  34.736347   
4           13.232697  34.298569  36.983864  35.270798  52.904499  34.771244   
5           13.259068  34.343376  36.983994  35.259872  52.806931  34.806145   

TeamID                                                  ...          1  \
PlayerID            2                     3             ...          6   
Attributes          x          y          x          y  ...          x   
frame                                                   ...              
1           52.224819  15.419998  40.538166  46.167290  ...  67.093147   
2           52.295338  15.411969  40.591766  46.167347  ...  67.093216   
3           52.365852  15.403940  40.645363  46.167404  ...  67.093277   
4           52.436363  15.395912  40.698959  46.167461  ...  67.093346   
5           52.506870  15.387884  40.752552  46.167519  ...  67.093407   

TeamID                                                                        \
PlayerID                       7                     8                     9   
Attributes          y          x          y          x          y          x   
frame                                                                          
1           15.537871  52.840797  43.730240  64.841843  45.449814  74.867973   
2           15.523928  52.836475  43.716923  64.841843  45.448952  74.863007   
3           15.509986  52.832150  43.703606  64.841850  45.448086  74.858047   
4           15.496042  52.827827  43.690292  64.841850  45.447227  74.853088   
5           15.482101  52.823502  43.676975  64.841858  45.446362  74.848114   

TeamID                      BALL             
PlayerID                    BALL             
Attributes          y          x          y  
frame                                        
1           37.785244  52.547379  34.098305  
2           37.775181  52.357082  34.115082  
3           37.765121  52.166763  34.131859  
4           37.755058  51.976418  34.148636  
5           37.744999  51.786053  34.165417  

[5 rows x 46 columns]